# Gaia Data

### On June 13, 2022 the [Gaia project](https://www.cosmos.esa.int/web/gaia/dr3) released is third major data release containg about 1.5 billion sources.

- For Astro 300, we will use a subset of the main data source. 

- This subset is still really large (1906.8 GB), so we will use python to access this data in an efficient manner.

- #### The Gaia database we will use is called `gaiadr3.gaia_source_lite`

In [ ]:
import numpy as np

from astropy.table import QTable
from astroquery.gaia import Gaia

---
# SQL/ADQL Database query language
 
SQL (Structured Query Language) is a language designed for managing data held in a relational database management systems. SQL has became the most widely used database language.

Astronomical Data Query Language (ADQL) is a specialised variant of SQL developed for use with the proliferation of astronomical datasets, and extends the functionality of SQL in an astronomical context.

## ADQL Query

A typical ADQL query has the form:

```
SELECT 
{columns}
FROM {database}
WHERE {conditions}
```

The ADQL commands are usually ALLCAPS and the other commands are lowercase.

There is a real example of a ADQL query to get the columns: `source_id`, `ra`, `dec`, and `parallax` from `gaiadr3.gaia_source_lite` database for all objects where the value of the `parallax` column is greater than 275 mas. The columns will be ordered by decreasing values of `parallax`:

```
SELECT TOP 100
source_id, ra, dec, parallax
FROM gaiadr3.gaia_source_lite
WHERE parallax > 275
ORDER BY parallax DESC
```

#### It is really good to add `TOP 100` to the `SELECT` when you first do a query, so you do not drop millions of lines into your notebook!

---
## Let's get some data

- First we create the query string as a doc-string

In [ ]:
query_one = """
SELECT TOP 100
source_id, ra, dec, parallax
FROM gaiadr3.gaia_source_lite
WHERE parallax > 275
ORDER BY parallax DESC
"""

In [ ]:
print(query_one)

## Submit our query to the Gaia archive server

In [ ]:
my_job_query = Gaia.launch_job(query_one)

### Check the status of the job

In [ ]:
print(my_job_query)

### Looks good so get the results

- The results will be a nice astropy Qtable

In [ ]:
my_parallax_table = my_job_query.get_results()

In [ ]:
my_parallax_table

---

# Getting data from a specific piece of the sky

### A very common search is to find objects within a certain angular distance from a point on the sky

<img src="https://uwashington-astro300.github.io/A300_images/Orion_Circle.png" width="400"/>


- The command `POINT(RA(deg), DEC(deg))` specifies a point on the celestial sphere.

- The command `DISTANCE(point1, point2)` computes the spherical angular distance between two points.

#### Here is a query to find all Gaia objects

- Within 0.5 degrees of RA = 90.0 deg, Dec = 10.0 deg 
- Brighter than 10th mag
- Have color (BP-RP) data.

```
SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE DISTANCE( POINT(90.0, 10.0), POINT(ra, dec) ) < 0.5
AND phot_g_mean_mag < 10.0
AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
```

- #### Extra `conditions` can be added with the `AND` command
- #### The `IS NOT NULL` command is very useful for ignoring rows with no data

In [ ]:
query_circle = """
SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE DISTANCE( POINT(90.0, 10.0), POINT(ra, dec) ) < 0.5
AND phot_g_mean_mag < 10.0
AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
"""

In [ ]:
my_job_query = Gaia.launch_job(query_circle)

In [ ]:
print(my_job_query)

In [ ]:
my_circle_table = my_job_query.get_results()

In [ ]:
my_circle_table

### Another common search is to find objects within a certain region bound by sets of `RA(deg)` and `DEC(deg)` coordinates

<img src="https://uwashington-astro300.github.io/A300_images/Orion_Square.png" width="400"/>


#### Here is a query to find all Gaia objects

- Within 0.5 degrees of RA = 90.0 deg, Dec = 10.0 deg 
- Brighter than 12th mag
- Have color (BP-RP) data.

```
SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE ra BETWEEN 89.0 AND 90.0
AND dec BETWEEN 8.0 AND 10.0 
AND phot_g_mean_mag < 12.0
AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
```

- #### The `BETWEEN` command is very useful for these sorts of searches

In [ ]:
query_square = """
SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE ra BETWEEN 89.0 AND 90.0
AND dec BETWEEN 8.0 AND 10.0 
AND phot_g_mean_mag < 12.0
AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
"""

In [ ]:
my_job_query = Gaia.launch_job(query_square)

In [ ]:
print(my_job_query)

### Notice that the number of objects is the same as your number limit (`SELECT TOP 100`).

- This is an indication that you have not found all of the objects that meet your criteria.
- Increase your search number limit (slowly) until the number of objects returned is less than you search number limit.

In [ ]:
query_square = """
SELECT TOP 300
source_id, ra, dec, phot_g_mean_mag, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE ra BETWEEN 89.0 AND 90.0
AND dec BETWEEN 8.0 AND 10.0 
AND phot_g_mean_mag < 12.0
AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
"""

In [ ]:
my_job_query = Gaia.launch_job(query_square)

In [ ]:
print(my_job_query)

### This is good since `<Table length=238>` is less than `SELECT TOP 300`

In [ ]:
my_square_table = my_job_query.get_results()

In [ ]:
my_square_table

### This is a long notebook, so you can use `.show_in_notebook()` to see the whole table

In [ ]:
my_square_table.show_in_notebook()

---
### The `BETWEEN` command is general. It can be used for more than just coordinates.

The image below is the color-magnitude diagram of all the stars from the Gaia observations. Notice that stellar types can be defined by the `BP-RP` color index. For example, [A-stars](https://en.wikipedia.org/wiki/A-type_main-sequence_star) have a `BP-RP` index between about -0.1 and 0.3.


<img src="https://uwashington-astro300.github.io/A300_images/Gaia_CMD_BW.png" width="400"/>

We can add the color-cut `bp_rp BETWEEN -0.1 AND 0.3` to only get data for A-stars.

In [ ]:
query_color = """
SELECT TOP 300
source_id, ra, dec, phot_g_mean_mag, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE ra BETWEEN 89.0 AND 90.0
AND dec BETWEEN 8.0 AND 10.0 
AND phot_g_mean_mag < 12.0
AND bp_rp BETWEEN -0.1 AND 0.3
ORDER BY bp_rp ASC
"""

In [ ]:
my_job_query = Gaia.launch_job(query_color)

In [ ]:
print(my_job_query)

In [ ]:
my_color_table = my_job_query.get_results()

In [ ]:
my_color_table

---
## Query Speed Considerations

- Making queries with lots of constraints can be really slow.
- It is often faster to make a broader (faster) query and then make subsets of the bigger table with masking.
- In this case I have removed the color cut (`AND bp_rp BETWEEN -0.1 AND 0.3`)

In [ ]:
query_faster = """
SELECT TOP 300
source_id, ra, dec, phot_g_mean_mag, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE ra BETWEEN 89.0 AND 90.0
AND dec BETWEEN 8.0 AND 10.0 
AND phot_g_mean_mag < 12.0
ORDER BY bp_rp ASC
"""

In [ ]:
my_job_query = Gaia.launch_job(query_faster)

In [ ]:
print(my_job_query)

In [ ]:
my_bigger_color_table = my_job_query.get_results()

### Make a subset of the big table

- Do the color cut here

In [ ]:
my_smaller_color_table = my_bigger_color_table[(my_bigger_color_table['bp_rp'] < 0.3) & 
                                               (my_bigger_color_table['bp_rp'] > -0.1)]

In [ ]:
my_smaller_color_table

---

## ADQL querys can get SUPER complicated! - I have shown you the merest baby steps. 

## If you want to see how the pros work, check out the [Gaia ADQL Guide](https://www.cosmos.esa.int/web/gaia-users/archive/writing-queries)